<a href="https://colab.research.google.com/github/budhrajaankita/202/blob/main/INFO202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil import parser
import pandas as pd
import nltk
import string
import os
import pandas as pd
import numpy as np
import time
import nltk
import re

nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
!pip install rank_bm25
from google.colab import drive

from IPython.core import display as ICD
from nltk.stem import WordNetLemmatizer
from rank_bm25 import BM25Okapi, BM25L, BM25Plus
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import json
# url_list = ["https://www.moneycontrol.com/news/tags/companies.html","https://www.moneycontrol.com/news/tags/moneycontrol-analysis.html","https://www.moneycontrol.com/news/business/moneycontrol-research/","https://www.moneycontrol.com/news/business/mutual-funds/","https://www.moneycontrol.com/news/business/personal-finance/","https://www.moneycontrol.com/news/tags/residential.html","https://www.moneycontrol.com/news/tags/commercial.html","https://www.moneycontrol.com/news/tags/urbanreforms.html","https://www.moneycontrol.com/news/tags/real-estate.html","https://www.moneycontrol.com/news/tags/auto.html","https://www.moneycontrol.com/news/politics/","https://www.moneycontrol.com/news/india/","https://www.moneycontrol.com/news/tags/bankingtech.html","https://www.moneycontrol.com/news/tags/insurancetech.html","https://www.moneycontrol.com/news/technology/"]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Web Crawling to get news related links/urls

In [20]:
url_list = []
def get_linked_urls(url, html):
        soup = BeautifulSoup(html, 'html.parser')
        for link in soup.find_all('a'):
            path = link.get('href')
            if "https://www.moneycontrol.com/news/" in str(path) and "html" in str(path):
              url_list.append(path)
              if len(url_list) == 10:
                break
            # if path and path.startswith('/'):
            #     path = url.join(url, path)
            # print(path)

url = "https://www.moneycontrol.com/news/"
htm = requests.get(url).text
get_linked_urls(url, htm)
print(url_list)

['https://www.moneycontrol.com/news/tags/companies.html', 'https://www.moneycontrol.com/news/tags/technical-analysis.html', 'https://www.moneycontrol.com/news/tags/currency.html', 'https://www.moneycontrol.com/news/tags/slideshows.html', 'https://www.moneycontrol.com/news/tags/mc-learn.html', 'https://www.moneycontrol.com/news/tags/education.html', 'https://www.moneycontrol.com/news/tags/science.html', 'https://www.moneycontrol.com/news/tags/books.html', 'https://www.moneycontrol.com/news/tags/mc-learn.html', 'https://www.moneycontrol.com/news/tags/company-fixed-deposits.html']


# Web Scraping to get news articles from fetched links/urls
max pages to scrape for each link is set to 3

In [21]:
news_articles = {}
main_news_articles = {}
news_count = 0 
main_json = {}

for main_url in url_list:
  p = 1
  max_pages = 3

  while p < max_pages:
    new_url = main_url+"/page-{}/".format(p)
    url = requests.get(new_url)
    documents = []
    print(new_url)
    data = url.text
    soup = BeautifulSoup(data,'html.parser')
    url_tag = soup.find('a',{'class':'act'})
    url_last = soup.find_all('a',{'class':'last'})
      
    articles = soup.find_all('li',{'class':'clearfix'})
    for article in articles:
      title = article.find('h2').text
      link = article.find('a').get('href')
      date = article.find('span').text
      a = parser.parse(date)
      new_a = a.replace(tzinfo=None)
      year = new_a.year
      news_response = requests.get(link, timeout=15)
      news_data = news_response.text
      news_soup = BeautifulSoup(news_data,'html.parser')

      if news_soup.find('div',{'class':'arti-flow'}):
          news_text = news_soup.find('div',{'class':'arti-flow'})
          for x in news_text.find_all("script"):
              x.decompose()
          for y in news_text.find_all('style'):
              y.decompose()
          try:
              news_text.find_all('a')[-1].decompose()
              news = news_text.text
          except IndexError:
              news = news_text.text
              
      news_articles[news_count] =  {"headline": title, "text": news, "year": str(year)}
      main_news_articles.update(news_articles)

      news_count+=1

    else:
      print('\nLimit set for maximum pages\n')
    p+=1
    
  print(len(main_news_articles))


https://www.moneycontrol.com/news/tags/companies.html/page-1/


/usr/local/lib/python3.8/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "



Limit set for maximum pages

https://www.moneycontrol.com/news/tags/companies.html/page-2/

Limit set for maximum pages

50
https://www.moneycontrol.com/news/tags/technical-analysis.html/page-1/

Limit set for maximum pages

https://www.moneycontrol.com/news/tags/technical-analysis.html/page-2/

Limit set for maximum pages

100
https://www.moneycontrol.com/news/tags/currency.html/page-1/

Limit set for maximum pages

https://www.moneycontrol.com/news/tags/currency.html/page-2/

Limit set for maximum pages

150
https://www.moneycontrol.com/news/tags/slideshows.html/page-1/

Limit set for maximum pages

https://www.moneycontrol.com/news/tags/slideshows.html/page-2/

Limit set for maximum pages

200
https://www.moneycontrol.com/news/tags/mc-learn.html/page-1/

Limit set for maximum pages

https://www.moneycontrol.com/news/tags/mc-learn.html/page-2/

Limit set for maximum pages

213
https://www.moneycontrol.com/news/tags/education.html/page-1/

Limit set for maximum pages

https://www.mon

In [24]:
mnc = pd.DataFrame.from_dict(main_news_articles, orient='index').drop_duplicates()
mnc = mnc.reset_index()
mnc

drive.mount('/content/drive')
%cd '/content/drive/MyDrive/'

# mnc.to_csv('mnc.csv')
mnc.to_csv('mnc.csv')
# print(news_count)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


# Transform and tokenize words

In [25]:
stop_words = set(stopwords.words('english'))
porter = PorterStemmer()

def word_transform(text):
  if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
  text = text.translate(str.maketrans('', '', string.punctuation))
  tokens = nltk.tokenize.word_tokenize(text)
  words = [word for word in tokens if word.isalpha()]
  words = [w for w in words if not w in stop_words]
  stemmed = [porter.stem(word) for word in words]
  return stemmed

# mnc_list = mnc.text.to_list()
# # print(mnc_list)
# list_tokenized = [word_transform(l) for l in mnc_list]
# print(list_tokenized)

# Calculate frequency of each word in all news articles


In [26]:
def get_list_tokenised(data):
  mnc_list = data.text.to_list()
  list_tokenized = [word_transform(l) for l in mnc_list]
  return list_tokenized

def calc_doc_freq(data):
  dfreq = {}
  list_tokenized = get_list_tokenised(data)
  for i in range(len(list_tokenized)):
      tokens = list_tokenized[i]
      for w in tokens:
          try:
              dfreq[w].add(i)
          except:
              dfreq[w] = {i}

  for i in dfreq:
      dfreq[i] = len(dfreq[i])

  return(dfreq)

In [27]:
dfreq = calc_doc_freq(mnc)
print(dfreq)

{'semiconductor': 5, 'vedanta': 2, 'group': 41, 'tuesday': 12, 'said': 180, 'sign': 25, 'pact': 2, 'japanes': 13, 'technolog': 55, 'compani': 94, 'develop': 79, 'indian': 76, 'glass': 2, 'display': 7, 'manufactur': 23, 'ecosystemth': 1, 'vedantaavanstr': 1, 'busi': 106, 'partner': 15, 'summit': 7, 'held': 61, 'last': 145, 'week': 89, 'tokyo': 3, 'japan': 17, 'the': 279, 'attend': 12, 'deleg': 1, 'global': 67, 'companiesth': 3, 'commit': 20, 'make': 134, 'india': 120, 'hub': 4, 'electron': 12, 'it': 183, 'focuss': 20, 'take': 104, 'lead': 47, 'creat': 59, 'industri': 53, 'ecosystem': 6, 'rather': 71, 'transfer': 9, 'md': 4, 'akarsh': 1, 'k': 10, 'hebbar': 1, 'saidth': 37, 'saw': 23, 'interest': 123, 'host': 12, 'eager': 5, 'part': 78, 'missionvedanta': 1, 'ltd': 19, 'arm': 15, 'resourc': 20, 'one': 194, 'world': 145, 'oil': 26, 'ga': 15, 'metal': 4, 'oper': 44, 'zinc': 2, 'silver': 5, 'copper': 2, 'iron': 6, 'ore': 4, 'steel': 4, 'aluminium': 2, 'power': 62, 'across': 73, 'south': 32, '

# New Calculate TFIDF for each word in each article

In [28]:
import numpy as np
from collections import Counter

def article_freq(word, dfreq):
      const = 0
      try:
          const = dfreq[word]
      except:
          pass
      return const

def cal_tfidf(dfreq, mnc):
  doc = 0
  tf_idf = {}
  len_mnc = len(mnc['text'])
  words_set = len(dfreq)
  names = [i for i in dfreq.keys()]
  print(len(names))
  tf_idf = pd.DataFrame(np.zeros((len_mnc, words_set)), columns= names)
  list_tokenized = get_list_tokenised(mnc)
  

  for i in range(len(mnc)):
      tokens = list_tokenized[i]
      counter = Counter(tokens)
      words_count = len(tokens)
      
      for token in np.unique(tokens):
        #term frequency
        tf = counter[token]/words_count
        #article frequency
        df = article_freq(token,dfreq)
        #inverse document frequency
        idf = np.log((len(mnc)+1)/(df+1))
        # print(idf)
        tf_idf[token][i] = tf*idf
        # print(tf_idf[token])
        # df_tf_idf[token][i] = df_tf[w][i] * idf[w]
      # doc += 1
  return tf_idf

In [29]:
tfidf_score = cal_tfidf(dfreq, mnc)
tfidf_score

13067


,semiconductor,vedanta,group,tuesday,said,sign,pact,japanes,technolog,compani,...,strategyanoth,onmani,consideredit,payout,reinvest,instrumentslook,changesa,frontarnav,pandyath,instrumentwel
0,0.161637,0.141614,0.021332,0.032829,0.007010,0.052067,0.09441,0.032102,0.037023,0.053319,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.033372,0.000000,0.010966,0.000000,0.00000,0.000000,0.000000,0.025024,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.038052,0.016250,0.000000,0.00000,0.000000,0.000000,0.015450,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.004983,0.000000,0.00000,0.000000,0.000000,0.009475,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.031382,0.000000,0.013750,0.000000,0.00000,0.000000,0.000000,0.026147,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.021812,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
365,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.063152,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
366,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.003952,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
367,0.000000,0.000000,0.000000,0.000000,0.000000,0.005954,0.00000,0.000000,0.000000,0.003048,...,0.011705,0.011705,0.011705,0.011705,0.011705,0.011705,0.011705,0.011705,0.011705,0.000000


# TF IDF vectorizer

# Cosine similarity of each article in the dataframe "mnc"

In [30]:

vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(mnc["text"])
start = time.time()
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("Time taken: %s seconds" % (time.time() - start))

h = [row for row in mnc.headline]
pretty_cos = pd.DataFrame(cosine_sim, columns = h, index = h)
pretty_cos.head()



Time taken: 0.02873516082763672 seconds


,Vedanta inks pacts with 30 Japanese firms to develop Indian semiconductor manufacturing ecosystem,"IL&FS resolves debt of Rs 56,943 crore, reduces number of entities to 101",Tata Motors to hike commercial vehicle prices by up to 2% from January,HDFC-HDFC Bank merger to shake up India’s debt market,Byju’s lenders seek quicker part-payment of $1.2 billion loan,Paytm’s surprise share buyback plan is drawing skepticism,KM Birla controlled AB Capital said to weigh selling insurance brokerage unit,Apcotex: Realisation to taper further; wait for value to emerge,Dalmia Bharat – Takeover of Jaypee's cement assets marks a new chapter on growth,"Can Avatar 2, one of the costliest-ever movie, be a box office winner?",...,Beyond Ponniyin Selvan: 6 other novels by Kalki Krishnamurthy,"October reading: New books from Sudha Murty, Orhan Pamuk and more authors",Eric Ambler’s spy thrillers still resonate today,Book excerpt: When Lala ji's whims crash against a principled consultant's analysis,Fed up with low bank FD rates? These corporate FDs offer up to 7.25% interest on three-year deposits,Senior citizen should carefully plan fixed income portfolio,Should you invest in corporate FDs for higher rates?,Things to consider while investing in fixed income products,Creating a vibrant fixed income portfolio,Advantages of debt funds over fixed deposits
Vedanta inks pacts with 30 Japanese firms to develop Indian semiconductor manufacturing ecosystem,1.000000,0.085587,0.095850,0.132259,0.133284,0.166794,0.137626,0.051200,0.068484,0.107594,...,0.102706,0.106865,0.141923,0.131955,0.149221,0.064335,0.168862,0.147854,0.127240,0.034883
"IL&FS resolves debt of Rs 56,943 crore, reduces number of entities to 101",0.085587,1.000000,0.114235,0.142958,0.211714,0.156977,0.136407,0.077907,0.069295,0.140603,...,0.103129,0.104763,0.130845,0.167260,0.162754,0.084662,0.176947,0.181384,0.145381,0.067544
Tata Motors to hike commercial vehicle prices by up to 2% from January,0.095850,0.114235,1.000000,0.187290,0.187122,0.199492,0.163619,0.057496,0.090546,0.155672,...,0.130628,0.143296,0.171209,0.171555,0.198571,0.075976,0.181719,0.239015,0.209617,0.063341
HDFC-HDFC Bank merger to shake up India’s debt market,0.132259,0.142958,0.187290,1.000000,0.233609,0.245330,0.182032,0.090176,0.124478,0.177643,...,0.159243,0.170740,0.206900,0.196039,0.247446,0.154320,0.240733,0.288855,0.260753,0.117781
Byju’s lenders seek quicker part-payment of $1.2 billion loan,0.133284,0.211714,0.187122,0.233609,1.000000,0.291854,0.312342,0.084105,0.118622,0.201411,...,0.180699,0.200841,0.231756,0.258803,0.258158,0.112957,0.260381,0.319822,0.259745,0.106205


# Linear Kernel
Since the tf-idf functionality in sklearn.feature_extraction.text produces normalized vectors, hence, in this case, cosine_similarity is equivalent to linear_kernel. (only slower in case of large datasets)

In [31]:

start = time.time()
linear_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print("Time taken: %s seconds" % (time.time() - start))

pretty_cos = pd.DataFrame(linear_sim, columns = h, index = h)
pretty_cos.head()


Time taken: 0.02819538116455078 seconds


,Vedanta inks pacts with 30 Japanese firms to develop Indian semiconductor manufacturing ecosystem,"IL&FS resolves debt of Rs 56,943 crore, reduces number of entities to 101",Tata Motors to hike commercial vehicle prices by up to 2% from January,HDFC-HDFC Bank merger to shake up India’s debt market,Byju’s lenders seek quicker part-payment of $1.2 billion loan,Paytm’s surprise share buyback plan is drawing skepticism,KM Birla controlled AB Capital said to weigh selling insurance brokerage unit,Apcotex: Realisation to taper further; wait for value to emerge,Dalmia Bharat – Takeover of Jaypee's cement assets marks a new chapter on growth,"Can Avatar 2, one of the costliest-ever movie, be a box office winner?",...,Beyond Ponniyin Selvan: 6 other novels by Kalki Krishnamurthy,"October reading: New books from Sudha Murty, Orhan Pamuk and more authors",Eric Ambler’s spy thrillers still resonate today,Book excerpt: When Lala ji's whims crash against a principled consultant's analysis,Fed up with low bank FD rates? These corporate FDs offer up to 7.25% interest on three-year deposits,Senior citizen should carefully plan fixed income portfolio,Should you invest in corporate FDs for higher rates?,Things to consider while investing in fixed income products,Creating a vibrant fixed income portfolio,Advantages of debt funds over fixed deposits
Vedanta inks pacts with 30 Japanese firms to develop Indian semiconductor manufacturing ecosystem,1.000000,0.085587,0.095850,0.132259,0.133284,0.166794,0.137626,0.051200,0.068484,0.107594,...,0.102706,0.106865,0.141923,0.131955,0.149221,0.064335,0.168862,0.147854,0.127240,0.034883
"IL&FS resolves debt of Rs 56,943 crore, reduces number of entities to 101",0.085587,1.000000,0.114235,0.142958,0.211714,0.156977,0.136407,0.077907,0.069295,0.140603,...,0.103129,0.104763,0.130845,0.167260,0.162754,0.084662,0.176947,0.181384,0.145381,0.067544
Tata Motors to hike commercial vehicle prices by up to 2% from January,0.095850,0.114235,1.000000,0.187290,0.187122,0.199492,0.163619,0.057496,0.090546,0.155672,...,0.130628,0.143296,0.171209,0.171555,0.198571,0.075976,0.181719,0.239015,0.209617,0.063341
HDFC-HDFC Bank merger to shake up India’s debt market,0.132259,0.142958,0.187290,1.000000,0.233609,0.245330,0.182032,0.090176,0.124478,0.177643,...,0.159243,0.170740,0.206900,0.196039,0.247446,0.154320,0.240733,0.288855,0.260753,0.117781
Byju’s lenders seek quicker part-payment of $1.2 billion loan,0.133284,0.211714,0.187122,0.233609,1.000000,0.291854,0.312342,0.084105,0.118622,0.201411,...,0.180699,0.200841,0.231756,0.258803,0.258158,0.112957,0.260381,0.319822,0.259745,0.106205


# *Get News Article Recommendation based on headline as input*

In [32]:
def get_news_recommendations(title, data, cosine_sim, indices):
    id = indices[title]
    sim_scores = list(enumerate(cosine_sim[id]))
    #Sort based on the sim scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar news headlines
    sim_scores = sim_scores[1:6]
    print(sim_scores)
    news_indices = [i[0] for i in sim_scores]
    # Get the top 10 most similar news headlines
    return pd.DataFrame(data.iloc[news_indices])

In [33]:
rec_vector = TfidfVectorizer(stop_words='english')

headlines = mnc['text'].drop_duplicates()

# Construct the TF-IDF matrix
tfidf_matrix = rec_vector.fit_transform(headlines.values.astype('U'))

# Generate the cosine similarity matrix
cosine_sim_headlines = cosine_similarity(tfidf_matrix, tfidf_matrix)

indices = pd.Series(mnc.index, index=mnc['headline']).dropna()

In [34]:
start = time.time()
cosine_recc = get_news_recommendations("Edtech’s failure is Indian education sector's curse to bear", mnc, cosine_sim_headlines, indices)
print("Time taken: %s seconds" % (time.time() - start))
cosine_recc

[(236, 0.1293683289936037), (224, 0.10725533778046283), (242, 0.10486166464792658), (238, 0.09437341492879854), (269, 0.09390293281059883)]
Time taken: 0.0022492408752441406 seconds


,index,headline,text,year
236,236,"DU admission: 72,865 candidates accept allotte...",...,2022
224,224,Adityanath govt's infrastructure push boosts e...,...,2022
242,242,Delhi University likely to open mid-entry wind...,...,2022
238,238,5G will take education to next level: PM Naren...,...,2022
269,269,Climate change contributed to deadly West Afri...,...,2022


In [35]:
start = time.time()
cosine_recc = get_news_recommendations("Indian rupee weakens past 81-mark for first time against US dollar", mnc, cosine_sim_headlines, indices)
print("Time taken: %s seconds" % (time.time() - start))
cosine_recc

[(130, 0.5902391458901699), (132, 0.17398104656906038), (121, 0.16706147861731313), (125, 0.12252706426541471), (129, 0.09680794150036391)]
Time taken: 0.0022020339965820312 seconds


,index,headline,text,year
130,130,Rupee hits fresh record low against US dollar,...,2022
132,132,U.S. dollar gains as Fed doubles down on tight...,...,2022
121,121,BRICS nations talk on reserve currency to redu...,...,2022
125,125,"Markets Podcast: Rupee may be at record low, b...",...,2022
129,129,Yen strengthens after Japan intervenes for fir...,...,2022


In [36]:
start = time.time()
cosine_recc = get_news_recommendations("Cummins India shares test 4-month high as it provides tech for Gail's hydrogen plant", mnc, cosine_sim_headlines, indices)
print("Time taken: %s seconds" % (time.time() - start))
cosine_recc

[(37, 0.1133747206686444), (16, 0.09235512525041156), (47, 0.0898831719793359), (32, 0.07725805296361811), (33, 0.07488820810835903)]
Time taken: 0.003190755844116211 seconds


,index,headline,text,year
37,37,Power giant NTPC will turn to nuclear in clima...,...,2022
16,16,"Deloitte CEO Punit Renjen to focus on India, n...",...,2022
47,47,Vedanta to raise up to Rs 500 crore via debent...,...,2022
32,32,Sun Pharma's Halol facility listed under USFDA...,...,2022
33,33,"Rockwell Automation expands to 3,000 people in...",...,2022


# Calculating BM25 scores for each article and giving output based on the score and exact matches

In [37]:
lemmatizer = WordNetLemmatizer()

def smart_tokenize(txt):
  text = txt.translate(str.maketrans('', '', string.punctuation))
  tokens = nltk.word_tokenize(text.lower())
  return [lemmatizer.lemmatize(w) for w in tokens]

def transform_text(text):
  if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
  text = text.translate(str.maketrans('', '', string.punctuation))
  tokens = nltk.tokenize.word_tokenize(text)
  words = [word for word in tokens if word.isalpha()]
  words = [w for w in words if not w in stop_words]
  stemmed = [porter.stem(word) for word in words]
  return stemmed

mnc['text_new'] = mnc.apply(lambda row: ' '.join(transform_text(row.text)), axis=1)
mnc.head()


,index,headline,text,year,text_new
0,0,Vedanta inks pacts with 30 Japanese firms to d...,...,2022,semiconductor vedanta group tuesday said sign ...
1,1,"IL&FS resolves debt of Rs 56,943 crore, reduce...",...,2022,ilf transport network ilf transport network ap...
2,2,Tata Motors to hike commercial vehicle prices ...,...,2022,tata motor trade watchlist portfolio messag se...
3,3,HDFC-HDFC Bank merger to shake up India’s debt...,...,2022,hdfc bank no pm strategi hold stock a sampl pm...
4,4,Byju’s lenders seek quicker part-payment of $1...,...,2022,a group creditor byju india valuabl startup as...


# BM25 on tokenized data and fetching feature names of article's text

In [38]:
tokenized_corpus = [doc.split(" ") for doc in mnc["text_new"]]
bm25 = BM25Okapi(tokenized_corpus)

vectorizer = TfidfVectorizer(use_idf=True)
vectors = vectorizer.fit_transform(mnc["text_new"])
words_set = vectorizer.get_feature_names_out()
print(words_set)


['aadmi' 'aakash' 'aalst' ... 'zoom' 'zte' 'zuckerberg']


## 1. Take input from user as a search query
## 2. Tokenize the input
## 3. Assign score to the query
## 4. Check for exact matches in article's text
## 5. Sort and display the result based on high BM25 scores

In [39]:

def smart_tokenize(txt):
  tokens = nltk.word_tokenize(txt.lower())
  return [lemmatizer.lemmatize(w) for w in tokens]

def exact_match(query, arr):
  count=0
  if type(query) == list:
    for i in query:
      if i.lower() in arr.lower():
        count += 1
        if count == len(query):
          return 1
      else:
        return 0
  elif query.lower() in arr.lower():
      return 1
  else:
    return 0
  if count>0:
    return count


def bm25_model(bm_query):
  for query in bm_query:
    print('Query:', query)
    t1 = time.time()
    tokenized_query = transform_text(query)
    print("tokenized_query: {}".format(tokenized_query))
    doc_scores = bm25.get_scores(tokenized_query)
    result = mnc[['headline','text', 'text_new', 'year']].copy()
    result['bm25score'] = pd.Series(doc_scores)
    tfid_query_vector = vectorizer.transform([x for x in tokenized_query])
    # print(vectors.shape)
    # print(tfid_query_vector.shape)
    result['exact'] = mnc.text_new.apply(lambda row: exact_match(tokenized_query, row))
    result.sort_values(['exact', 'bm25score'], ascending=[False, False], inplace=True)
    final_result = result[['headline','text','text_new', 'year', 'bm25score', 'exact']]
    ICD.display(final_result[final_result['bm25score'] > 0])
    t2 = time.time() - t1

  print('elapsed:', t2, 's')

In [40]:
bm_query = [input('Enter search query:')]
bm25_model(bm_query)


Enter search query:fixed deposit india
Query: fixed deposit india
tokenized_query: ['fix', 'deposit', 'india']


,headline,text,text_new,year,bm25score,exact
366,Things to consider while investing in fixed in...,...,in time crisi alway question investor park mon...,2012,5.599023,1
210,MC Learn | Can I get rich fast by investing in...,...,repres imag chapter in chapter commonli ask qu...,2022,5.556229,1
239,"In Kolkata’s Sonagachi, sex workers are groomi...",...,in kolkata neighbour district estim lakh sex w...,2022,1.658242,1
364,Senior citizen should carefully plan fixed inc...,...,anil choprabajaj capitalfix incom instrument p...,2015,6.734399,0
367,Creating a vibrant fixed income portfolio,...,creation vibrant strong fix incom portfolio im...,2012,6.211982,0
...,...,...,...,...,...,...
206,MC Learn Fundamental Investing | Active Invest...,...,modul chapter you alway activ look invest idea...,2022,0.519314,0
123,World economy roiled by simultaneous shocks ec...,...,the world economi show sign rapid downshift co...,2022,0.506330,0
124,Watch | How Rupee's Fall Affects Stock Market ...,...,the world economi show sign rapid downshift co...,2022,0.506330,0
305,Olympic skiers and snowboarders are competing ...,...,itali loui phillip vito iii fall compet men ha...,2022,0.485328,0


elapsed: 0.04355669021606445 s


# Using get_top_n from BM25 module based on user input

In [41]:

query2 = input('Enter search query:')
tokenized_query = transform_text(query2)
print(tokenized_query)
docs = bm25.get_top_n(tokenized_query, mnc["text_new"], n=5)
df_search = mnc[mnc["text_new"].isin(docs)]
df_search.head()


Enter search query:fixed deposit india
['fix', 'deposit', 'india']


,index,headline,text,year,text_new
364,377,Senior citizen should carefully plan fixed inc...,...,2015,anil choprabajaj capitalfix incom instrument p...
365,378,Should you invest in corporate FDs for higher ...,...,2013,high inflat eat save broader level keep intere...
366,379,Things to consider while investing in fixed in...,...,2012,in time crisi alway question investor park mon...
367,380,Creating a vibrant fixed income portfolio,...,2012,creation vibrant strong fix incom portfolio im...
368,381,Advantages of debt funds over fixed deposits,...,2011,what option person choos invest debt instrumen...


# Testing the same model on another annotated dataset from Kaggle.

#Movie Dataset Recommendation


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


#Dataset taken from https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/ [License CC0: Public Domain]
Saved in Google drive

In [6]:
meta = pd.read_csv("movies_metadata.csv")
meta = meta[(meta['title'].notna()) & (meta['overview'].notna())]
meta = meta.drop_duplicates(subset="title", keep="first")
old_metadata = meta.reset_index()
metadata = old_metadata[1:10001]
metadata

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
1,1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,10314,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,74840,tt0113918,es,Nadie hablará de nosotras cuando hayamos muerto,...,1995-09-16,0.0,104.0,"[{'iso_639_1': 'es', 'name': 'Español'}]",Released,NaN,Nobody Will Speak of Us When We're Dead,False,7.4,8.0
9997,10315,False,"{'id': 2396, 'name': 'Asterix and Obelix Colle...",0,"[{'id': 10751, 'name': 'Family'}, {'id': 12, '...",NaN,9564,tt0133385,fr,Astérix & Obélix contre César,...,1999-02-01,1644060.0,109.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,The Gauls are revolting!,Asterix & Obelix Take on Caesar,False,5.9,409.0
9998,10316,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,102081,tt0051816,en,The Key,...,1958-05-28,0.0,134.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,He had the life expectancy of a match.,The Key,False,6.4,8.0
9999,10317,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",NaN,21290,tt0374248,en,Strings,...,2004-10-24,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,We are all connected.,Strings,False,7.1,18.0


In [7]:
metadata.loc[metadata["title"] == "Pokémon: Spell of the Unknown"]

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
4053,4114,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991,tt0235679,ja,Pokémon 3: The Movie,...,2000-07-08,68411275.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,143.0


In [8]:
# metadata['title'] = metadata['title'].drop_duplicates()
indices = pd.Series(metadata.index, index=metadata['title']).dropna()
indices

title
Jumanji                                        1
Grumpier Old Men                               2
Waiting to Exhale                              3
Father of the Bride Part II                    4
Heat                                           5
                                           ...  
Nobody Will Speak of Us When We're Dead     9996
Asterix & Obelix Take on Caesar             9997
The Key                                     9998
Strings                                     9999
Kin-dza-dza!                               10000
Length: 10000, dtype: int64

In [14]:
movie_plots = metadata['overview']
print(movie_plots)
tfidf = TfidfVectorizer()

# Construct the TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(movie_plots.values.astype('U'))

start = time.time()

# Generate the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

print("Time taken: %s seconds" % (time.time() - start))

print(cosine_sim)



1        When siblings Judy and Peter discover an encha...
2        A family wedding reignites the ancient feud be...
3        Cheated on, mistreated and stepped on, the wom...
4        Just when George Banks has recovered from his ...
5        Obsessive master thief, Neil McCauley leads a ...
                               ...                        
9996     Escaping gangsters trying to kill her because ...
9997     Set in 50 B.C., Asterix and Obelix are living ...
9998     In wartime England, circa 1941, poorly-armed t...
9999     Strings is a mythological story about the son ...
10000    The story takes place on the desert planet "Pl...
Name: overview, Length: 10000, dtype: object
Time taken: 4.951331853866577 seconds
[[1.         0.05818997 0.03903764 ... 0.02445126 0.0217869  0.04776456]
 [0.05818997 1.         0.045941   ... 0.00895019 0.02084109 0.01942385]
 [0.03903764 0.045941   1.         ... 0.02689681 0.04733199 0.04817282]
 ...
 [0.02445126 0.00895019 0.02689681 ... 1.   

In [17]:
start = time.time()
cosine_org_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_org_sim)
print("Time taken: %s seconds" % (time.time() - start))

[[1.         0.05818997 0.03903764 ... 0.02445126 0.0217869  0.04776456]
 [0.05818997 1.         0.045941   ... 0.00895019 0.02084109 0.01942385]
 [0.03903764 0.045941   1.         ... 0.02689681 0.04733199 0.04817282]
 ...
 [0.02445126 0.00895019 0.02689681 ... 1.         0.02882221 0.02890602]
 [0.0217869  0.02084109 0.04733199 ... 0.02882221 1.         0.04458356]
 [0.04776456 0.01942385 0.04817282 ... 0.02890602 0.04458356 1.        ]]
Time taken: 4.670016527175903 seconds


In [18]:
def get_recommendations(title, cosine_sim, indices):
    idx = indices[title]
    print(idx)
    # print(cosine_sim[idx])
    sim_scores = list(enumerate(cosine_sim[idx]))
    print(sim_scores)
    #Sort based on the sim scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar movies
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    # Get the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

start = time.time()
print(get_recommendations("Strings", cosine_sim, indices))
print("Time taken: %s seconds" % (time.time() - start))

9999
[(0, 0.04776455758089371), (1, 0.019423848016761395), (2, 0.04817281803229344), (3, 0.03869991602377959), (4, 0.04190690498176327), (5, 0.01072079745416054), (6, 0.022788024786669164), (7, 0.024218093268045804), (8, 0.03616223658432582), (9, 0.005540911508667604), (10, 0.025028974539589303), (11, 0.024179459392438634), (12, 0.029266702996552554), (13, 0.07192872343272119), (14, 0.01685357771328952), (15, 0.026843291389937813), (16, 0.04532075631843496), (17, 0.049812653677110144), (18, 0.008178643869373696), (19, 0.027574844933598342), (20, 0.03176580128805829), (21, 0.028446304547230204), (22, 0.01532703040496488), (23, 0.025054218139480403), (24, 0.02862826225915515), (25, 0.024660943233690488), (26, 0.019408471681381583), (27, 0.014189171797801029), (28, 0.043118954006319585), (29, 0.022846745255733387), (30, 0.046243264829780056), (31, 0.03807148920603467), (32, 0.033670895419103425), (33, 0.024856417347266624), (34, 0.050383634121987916), (35, 0.017288532963970035), (36, 0.03

In [22]:
def get_recommendations(title, cosine_org_sim, indices):
    idx = indices[title]
    print(idx)
    # print(cosine_sim[idx])
    sim_scores = list(enumerate(cosine_org_sim[idx]))
    print(sim_scores)
    #Sort based on the sim scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores for 10 most similar movies
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    # Get the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

start = time.time()
print(get_recommendations("15 Minutes", cosine_org_sim, indices))
print("Time taken: %s seconds" % (time.time() - start))

3981
[(0, 0.03489569301869565), (1, 0.013264659392281164), (2, 0.0287269762175595), (3, 0.0425526557277802), (4, 0.007565147651980887), (5, 0.024218337377094244), (6, 0.016926476006113283), (7, 0.01544454114661028), (8, 0.013279038117156071), (9, 0.02427705301027306), (10, 0.051319696467075404), (11, 0.02029398235887929), (12, 0.040164879567380136), (13, 0.021397021678605927), (14, 0.0189021653644615), (15, 0.018631529034045975), (16, 0.03640799411470748), (17, 0.039214409877618456), (18, 0.016956936215723948), (19, 0.027999820842187505), (20, 0.008146915239992315), (21, 0.051918858893086346), (22, 0.023652553609300863), (23, 0.03398676781118551), (24, 0.08408731938680541), (25, 0.010742661686346034), (26, 0.03795570045491939), (27, 0.014522076545245031), (28, 0.03258602329711677), (29, 0.04801155197314108), (30, 0.04274936660738), (31, 0.047968602903549186), (32, 0.02323517556484275), (33, 0.05607610705526234), (34, 0.07516975743519369), (35, 0.010268718248286166), (36, 0.058177884681